In [1]:
from view_rides import add_path, show_path
import pandas as pd
import sys
sys.path.extend(['..', '../..'])
import psycopg2
from postgis.psycopg import register
from db_importer.settings import *
from adsp.db_utils import build_and_execute_query
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib as mpl
from matplotlib import ticker
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import contextily as cx

class DatabaseConnection(object):
    def __enter__(self):
        self.conn = psycopg2.connect(f"dbname='{DB_NAME}' user='{DB_USER}' password='{DB_PASSWORD}' host='{DB_HOST}' port='{DB_PORT}'")
        self.conn.autocommit = True

        register(self.conn)
        self.cur = self.conn.cursor()

        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_tb is None:
            self.conn.commit()
            self.cur.close()
            self.conn.close()
        else:
            self.conn.rollback()
            self.cur.close()
            self.conn.close()

In [ ]:
#filename = 'VM2_-186381149'

In [23]:
def get_dataframe(start, end, exclude = (0, 0, 0, 0)):
    res = build_and_execute_query(start, end, exclude)
    df = pd.DataFrame(res, columns=['filename', 'coords', 'velo', 'dur', 'dist', 'timestamp', 'min_ts', 'max_ts', 'time_diff', 'avg_v', 'group'])

    df['rad'] = 1

    df = df[~(df.rad.isnull() | df.timestamp.isnull() | df.coords.isnull())]

    df['lon'], df['lat'] = zip(*df.coords.values)

    return df

In [27]:
def analyze_scenario(df):

    #plt.xlim(0, 200)

    df2 = df[~df.velo.isna()]

    xmin = df2['lon'].min()
    xmax = df2['lon'].max()
    ymin = df2['lat'].min()
    ymax = df2['lat'].max()

    df_grouped = df2.groupby('filename', sort=False)

    def interact_plot(val):
        fig, ax = plt.subplots(1, 2, figsize=[30, 10])
        #ax = plt.axes()
        ax[0].grid()
        ax[0].xaxis.set_major_locator(ticker.LinearLocator(numticks=20))
        ax[0].set_ylabel('Speed (Km/h)')
        ax[0].set_xlabel('Time (timestamp / 1e9 -> seconds)')
        for ride_group_name in list(df_grouped.groups.keys())[val-1:val]:
            df_ride_group = df_grouped.get_group(ride_group_name)
            points = np.array([df_ride_group['timestamp'].map(lambda v: (v.value - df_ride_group['timestamp'].iloc[0].value) / 1e9), df_ride_group['velo'] * 3.6]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            lc = LineCollection(segments, cmap='copper')
            lc.set_array(df_ride_group['velo'] * 3.6)
            lc.set_linewidth(df_ride_group['velo']* 3.6 + 1)
            ax[0].add_collection(lc)
            ax[0].autoscale()
            #ax[0].plot(df_ride_group['timestamp'].map(lambda v: (v.value - df_ride_group['timestamp'].iloc[0].value) / 1e9), df_ride_group['velo'] * 3.6, label="One ride")
            points = np.array([df_ride_group['lon'], df_ride_group['lat']]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            lc = LineCollection(segments, cmap='copper')
            lc.set_array(df_ride_group['velo'] * 3.6)
            lc.set_linewidth(df_ride_group['velo']* 3.6 + 1)
            ax[1].add_collection(lc)
            ax[1].autoscale()
            #ax[1].plot(df_ride_group['lon'], df_ride_group['lat'])
        ax[1].set_xlim(xmin = xmin, xmax = xmax)
        ax[1].set_ylim(ymin = ymin, ymax = ymax)
        cx.add_basemap(ax[1], crs='EPSG:4326', source=cx.providers.Esri.WorldImagery)
        ax[1].set_aspect(1.7)

    interact(interact_plot, val=(1,len(df_grouped),1))

In [25]:
df = get_dataframe((13.416169,52.521508,13.416513,52.521832), (13.41591,52.522275,13.416421,52.522569), (13.415467,52.521919,13.416219,52.522107))

In [28]:
analyze_scenario(df)

interactive(children=(IntSlider(value=155, description='val', max=309, min=1), Output()), _dom_classes=('widge…

In [ ]:
df_grouped = df.groupby('filename', sort=False)
for i, ride_group_name in enumerate(df_grouped.groups):
    df_ride_group = df_grouped.get_group(ride_group_name)
    add_path(df_ride_group, lod=1)

show_path()